# Worksheet 08

Name: Ruihang Liu
UID: U49596386

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

Trying to figure out what species does this dianasour belong to based on its weight. To do that we need:

$Baye's Rule: Pr(S_i | X_j) = \frac{P(X_i|S_j)P(S_j)}{P(X_i)}$

* $P(S_j)$ is the prior probability of seeing species $S_j$ (that probability would be higher for the Stegosauruses than the Raptors for example). 
* $P(X_i | S_j)$ is the **PDF** of species $S_j$ weights evaluated at weight $X_i$ (seeing a Sauropod that weighs 100 tons is way more likely than seeing a Raptor that weighs 100 tons)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.229091762575663, 5.170364848697103, 5.226742914022274, 7.5435729935589855, 4.418031171433688, 5.00661827071544, 4.855897046499944, 5.042059952170689, 6.11459041962991, 3.5764282007386003]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8 , 1 , 10 ).tolist()
print(c2)

[7.155292979375972, 6.846941952261757, 6.975779026602272, 7.258233085259429, 7.5369406404855495, 9.051699741279403, 8.9771596252398, 8.94720955166336, 10.595528963492638, 8.465420218494923]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[3.5764282007386003, 6.11459041962991, 5.042059952170689, 4.855897046499944, 5.00661827071544, 8.465420218494923, 4.418031171433688, 7.5435729935589855, 5.226742914022274, 5.170364848697103]


Mixture Model:
* X comes from a mixture model with k mixture components if the prob distribution of X is $P(X = x) = \sum_{j=1}^{k}P(C_j)P(X = x | C_j)$

c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

For each individual probability's
1. mean
2. variance

And a probaility of x being i-th species (in our example, it's 1/2 - coin toss)

**Gaussian Mixture Model**: when all $P(X | S_j)$ are normal distribution with parameters:

1. mean
2. std

Expectation Maximization Algorithm (EM):

1. Start with random $\mu, \sum, P(S_j)$
2. Compute $P(C_j | X_I)$ for all Xi by using $\mu, \sum, P(S_j)$
3. Compute / Update $\mu, \sum, P(S_j)$ from $P(C_j | X_I)$
4. Repeat 2 & 3 until convergence

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)  ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.5764282007386003, 6.11459041962991, 5.042059952170689, 4.855897046499944, 5.00661827071544, 4.418031171433688, 5.226742914022274, 5.170364848697103]
[8.465420218494923, 7.5435729935589855]
P(C_1) = 0.8,  P(C_2) = 0.2
mean_1 = 4.926341602988456,  mean_2 = 8.004496606026954
var_1 = 0.4583958432532104,  var_2 = 9.687488799261459


Means are pretty close. But variance is a bit off.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  3.5764282007386003
probability of observing that point if it came from cluster 0 =  0.01138990036621079
probability of observing that point if it came from cluster 1 =  0.03709622474735474
point =  6.11459041962991
probability of observing that point if it came from cluster 0 =  0.030238506140661366
probability of observing that point if it came from cluster 1 =  0.040404936856384466
point =  5.042059952170689
probability of observing that point if it came from cluster 0 =  0.8430071617599713
probability of observing that point if it came from cluster 1 =  0.039300003419783305
point =  4.855897046499944
probability of observing that point if it came from cluster 0 =  0.8600845786463225
probability of observing that point if it came from cluster 1 =  0.039062520339641235
point =  5.00661827071544
probability of observing that point if it came from cluster 0 =  0.8570570690023374
probability of observing that point if it came from cluster 1 =  0.039255797612375874
point =  8.465

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7219944170282508,  P(C_2) = 0.27800558297174904
mean_1 = 4.969241139253586,  mean_2 = 7.029384889678809
var_1 = 0.34280856199993603,  var_2 = 2.9705130816270593


Pretty different. Now it's much more likely to be classified to C2. And the variable is much lower than k-mean.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  3.5764282007386003
probability of observing that point if it came from cluster 0 =  0.0003028952168903071
probability of observing that point if it came from cluster 1 =  0.06833913165581684
point =  6.11459041962991
probability of observing that point if it came from cluster 0 =  0.0043842000681968505
probability of observing that point if it came from cluster 1 =  0.12808099837409262
point =  5.042059952170689
probability of observing that point if it came from cluster 0 =  1.1377854836893446
probability of observing that point if it came from cluster 1 =  0.10737097698133535
point =  4.855897046499944
probability of observing that point if it came from cluster 0 =  1.101844101968907
probability of observing that point if it came from cluster 1 =  0.1027602513411218
point =  5.00661827071544
probability of observing that point if it came from cluster 0 =  1.1568497582289516
probability of observing that point if it came from cluster 1 =  0.1065097552273294
point =  8.4654202

It becomes just a bit more certain that it comes from C_1.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
assignments = [[], []]
for x in zip(data, prob_c0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [5.042059952170689, 4.855897046499944, 5.00661827071544, 4.418031171433688, 5.226742914022274, 5.170364848697103]
points in cluster 1 : [3.5764282007386003, 6.11459041962991, 8.465420218494923, 7.5435729935589855]
